Create the environment as this before running the kernel and selecting that enviroinment
conda create -n venv_unfaro python=3.10
source activate venv_unfaro

In [ ]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()


In [ ]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [ ]:
documents

In [ ]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [ ]:
response=query_engine.query("What is attention is all yopu need?")
print(response)

In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

In [ ]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)